Tworzenie samej bazy danych

In [ ]:
CREATE DATABASE Projekt_BD

Tworzenie wszystkich tabel w bazie danych

In [ ]:
-- Tworzenie bazy danych
DROP TABLE naprawy;
DROP TABLE pracownicy;
DROP TABLE Adresy;
DROP TABLE Historia_pracownikow;
DROP TABLE Klienci;
DROP TABLE Kontakty;
DROP TABLE Urzadzenia;
DROP TABLE Urzadzenia_klientow;


-- Tabela Pracownicy
CREATE TABLE Pracownicy (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    nazwisko VARCHAR(255) NOT NULL,
	imie VARCHAR(255) NOT NULL
);


-- Tabela Klienci
CREATE TABLE Klienci (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    nazwa_firmy VARCHAR(255) NOT NULL
);

-- Tabela osób do kontaktu z klientami
CREATE TABLE Kontakty (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_firmy INTEGER REFERENCES Klienci(id),
	nazwisko VARCHAR(255) NOT NULL,
	imie VARCHAR(255) NOT NULL,
	tel_kierunkowy VARCHAR(3),
	telefon INTEGER,
	email VARCHAR(255)
);

-- Tabela adresów klientów
CREATE TABLE Adresy (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_klienta INTEGER REFERENCES Klienci(id),
	ulica VARCHAR(255) NOT NULL,
	numer_budynku VARCHAR(15) NOT NULL,
	numer_lokalu VARCHAR(15),
	miasto VARCHAR(255) NOT NULL,
	kod_pocztowy VARCHAR(15)
);

-- Tabela Urzadzenia
CREATE TABLE Urzadzenia (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    producent VARCHAR(255) NOT NULL,
    model VARCHAR(255) NOT NULL
);

-- Tabela z urzadzeniami klientów
CREATE TABLE Urzadzenia_klientow (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_modelu INTEGER REFERENCES Urzadzenia(id),
	numer_seryjny VARCHAR(255) NOT NULL
);

-- Tabela Naprawy
CREATE TABLE Naprawy (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    nr_naprawy INTEGER NOT NULL,
    pracownik_id INTEGER REFERENCES Pracownicy(id),
    klient_id INTEGER REFERENCES Klienci(id),
    urzadzenie_id INTEGER REFERENCES Urzadzenia(id),
    opis_klienta TEXT,
    status VARCHAR(50) NOT NULL,
    diagnoza TEXT,
    wykonane_czynnosci TEXT,
    czesci_do_wymiany TEXT,
    data_przyjecia DATE NOT NULL,
    data_rozpoczecia DATE,
    data_zakonczenia DATE
);

-- Tabela Historia pracowników
-- Tabela rejestrująca wszsystkich pracownikow biorących udział w naprawie
CREATE TABLE Historia_pracownikow (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_naprawy INTEGER REFERENCES Naprawy(id),
	id_pracownika INTEGER REFERENCES Pracownicy(id)
);

-- 1. Zbieranie wszystkich napraw
SELECT * FROM Naprawy;

-- 2. Śledzenie ilości napraw w miesiącu
SELECT COUNT(*), strftime('%Y-%m', data_przyjecia) as Miesiac
FROM Naprawy
GROUP BY Miesiac;

-- 3. Ilość napraw dokonywanych przez każdego technika indywidualnie
SELECT pracownik_id, nazwisko, COUNT(*) as liczba_napraw
FROM Naprawy
JOIN Pracownicy ON Pracownicy.id = Naprawy.pracownik_id
GROUP BY pracownik_id;

-- 4. Wyszukiwanie napraw po kliencie i/lub numerze naprawy
-- Zmienne: @nazwa_firmy, @nr_naprawy
SELECT * FROM Naprawy
JOIN Klienci ON Klienci.id = Naprawy.klient_id
WHERE (nazwa_firmy = @nazwa_firmy OR @nazwa_firmy IS NULL)
AND (nr_naprawy = @nr_naprawy OR @nr_naprawy IS NULL);

-- 5. Sprawdzenie urządzeń, których producenta przychodzi najwięcej
SELECT producent, COUNT(*) as liczba_napraw
FROM Naprawy
JOIN Urzadzenia ON Urzadzenia.id = Naprawy.urzadzenie_id
GROUP BY producent
ORDER BY liczba_napraw DESC;

-- 6. Naprawy oczekujące po przyjęciu oraz w trakcie naprawy
SELECT * FROM Naprawy
WHERE status IN ('urządzenie przyjęte', 'w trakcie naprawy')
ORDER BY data_przyjecia ASC, data_rozpoczecia ASC;


Uzupełnienie tabel danymi